# Twitter Sentiment Analysis - POC
---

## 4. Cleaning Pipeline

So far the steps have been the same as they'd have been for the main project. Settting up a cleaning pipeline is an involved process and the formal beginning of the POC. For the final project, I will not be sampling the data down and all the processing steps will be encapsulated in a custom module.


## POC Only - Sample Data

In [1]:
import re
import os
import time
import json

import numpy as np
import pandas as pd

import urlextract
from html import unescape
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from collections import Counter
from sklearn.base import BaseEstimator, TransformerMixin

In [3]:
# load minimally prepared X, y train subsets
deduped_path = os.path.join("..","data","1.2_deduped","tweets","prepared")
X_train = pd.read_csv(os.path.join(deduped_path, "X_train.csv"))
y_train = pd.read_csv(os.path.join(deduped_path, "y_train.csv"))

In [10]:
# sample down considerably to X, y sample subsets
from sklearn.model_selection import train_test_split

X, X_rest, y, y_rest = train_test_split(X_train, y_train, test_size=0.999, random_state=159)

The plan is to forget about the `_rest` datasets and focus on the X, y small subsets, as if they were the entire training data.

In [11]:
print(f'Dataset size: {len(X):0.0f}')
print(f'Target distribution: {sum(y["target"])/len(y):0.3f}')

Dataset size: 779
Target distribution: 0.529


In [12]:
X.head(5)

,ID,username,tweet
401292,1307540808128700417,D35NUD4T3,Bionic themed game. Let’s play Fighters! 😉😈 ht...
414709,1314591505466626049,halfmoonbean,the amt of pictures mj posted today... \nwe lo...
444919,1308438100700524546,devilsvalentine,@ThrowAwayPogue i’m so proud 🥺
722403,1308088080658608131,ONLY4TREASURE,@hwankyuthree what’s happening 🥺
334176,1310026458740465671,mentalninja33,@TheEdumicator @unearthvii It really is a mast...


We need to create an array out of the tweet column, and that array will not contain the shuffled indices in a randomly sampled training dataset so we capture that in a column:

In [13]:
X.insert(3, 'index', X.index)
X.index = range(len(X))
X.head()

,ID,username,tweet,index
0,1307540808128700417,D35NUD4T3,Bionic themed game. Let’s play Fighters! 😉😈 ht...,401292
1,1314591505466626049,halfmoonbean,the amt of pictures mj posted today... \nwe lo...,414709
2,1308438100700524546,devilsvalentine,@ThrowAwayPogue i’m so proud 🥺,444919
3,1308088080658608131,ONLY4TREASURE,@hwankyuthree what’s happening 🥺,722403
4,1310026458740465671,mentalninja33,@TheEdumicator @unearthvii It really is a mast...,334176


In [14]:
# create an array of Tweets
X_array = np.array(X.iloc[:, 2]).ravel()
X_array[:2]

array(['Bionic themed game. Let’s play Fighters! 😉😈 https://t.co/wNGLFlmz4C',
       'the amt of pictures mj posted today... \nwe love kim myungjun🥺❤️'],
      dtype=object)

In [15]:
y[:2]

,target
401292,1
414709,1


### Acknowledgements

The contraction map and `expand_contractions` function were adapted from this [KDnuggests article.](https://www.kdnuggets.com/2018/08/practitioners-guide-processing-understanding-text-2.html)

The **DocumentToWordCounterTransformer** class was inspired by Aurelien Geron's **EmailToWordCounterTransformer** class from his famous [classification notebook](https://github.com/ageron/handson-ml/blob/master/03_classification.ipynb), and the **WordCounterToVectorTransformer** is a straight copy of Geron's same class.

In [16]:
def expand_contractions(text, contractions_map):
    
    pattern = re.compile('({})'.format('|'.join(contractions_map.keys())), 
                        flags=re.IGNORECASE|re.DOTALL)
    
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contractions_map.get(match)\
                                if contractions_map.get(match)\
                                else contractions_map.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def is_ascii(doc):
    try:
        doc.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
           
class DocumentToWordCounterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, expand_contractions=True, lower_case=True, 
                 replace_usernames=True, unescape_html=True, 
                 replace_urls=True, replace_numbers=True, 
                 remove_junk=True, remove_punctuation=True, 
                 replace_emojis=True, replace_nonascii=True, 
                 remove_stopwords=True, lemmatization=True):
        self.expand_contractions = expand_contractions
        self.lower_case = lower_case
        self.replace_usernames = replace_usernames
        self.unescape_html = unescape_html
        self.replace_urls = replace_urls
        self.replace_numbers = replace_numbers
        self.remove_junk = remove_junk
        self.remove_punctuation = remove_punctuation
        self.replace_emojis = replace_emojis
        self.replace_nonascii = replace_nonascii
        self.remove_stopwords = remove_stopwords
        self.lemmatization = lemmatization
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X_transformed = []
        for doc in X:
            if self.lower_case:
                doc = doc.lower()
            if self.expand_contractions and contractions_map is not None:
                doc = expand_contractions(doc, contractions_map)
            if self.replace_usernames:
                doc = re.sub(r'^@([^\s]+)',' USERNAME ', doc)
            if self.unescape_html:
                doc = unescape(doc)
            if self.replace_urls and url_extractor is not None:
                urls = list(set(url_extractor.find_urls(doc)))
                urls.sort(key=lambda url: len(url), reverse=True)
                for url in urls:
                    doc = doc.replace(url, ' URL ')
            if self.replace_numbers:
                doc = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', ' NUMBER ', doc)
            if self.remove_junk:
                pattern = r'\¥|\â|\«|\»|\Ñ|\Ð|\¼|\½|\¾|\!|\?|\¿|\x82\
                            |\x83|\x84|\x85|\x86|\x87|\x88|\x89|\
                            |\x8a|\x8b|\x8c|\x8d|\x8e|\°|\µ|\´|\º|\¹|\³'
                doc = re.sub(pattern,'', doc)
            if self.remove_punctuation:
                doc = re.sub(r'\W+', ' ', doc, flags=re.M)
            if self.replace_emojis:
                doc = re.sub(r'[^\x00-\x7F]+', ' EMOJI ', doc)
            if self.replace_nonascii:
                if is_ascii(doc) == False:
                    doc = ' NONASCII '
            word_counts = Counter(doc.split())
            if self.remove_stopwords:
                # 25 semantically non-selective words from the Reuters-RCV1 dataset
                stop_words = ['a','an','and','are','as','at','be','by','for','from',
                              'has','he','in','is','it','its','of','on','that','the',
                              'to','was','were','will','with']
                for word in stop_words:
                    try:
                        word_counts.pop(word)
                    except KeyError:
                        continue
            if self.expand_contractions:
                    leftovers = ['t','s','d','m','ve','re','ll','']
            if self.lemmatization and lemmatizer is not None:
                lemmatized_word_counts = Counter()
                for word, count in word_counts.items():
                    lemmatized_word = lemmatizer.lemmatize(word)
                    lemmatized_word_counts[lemmatized_word] += count
                word_counts = lemmatized_word_counts      
            X_transformed.append(word_counts)
        return np.array(X_transformed)

In [18]:
with open("contractions_map.json") as f:
    contractions_map = json.load(f)

url_extractor = urlextract.URLExtract()
lemmatizer = WordNetLemmatizer()

In [19]:
X_wordcounts = DocumentToWordCounterTransformer().fit_transform(X_array)

In [20]:
for counter in X_wordcounts[2:5]:
    counter = str(counter).split("(")[1]
    print(counter)

{'USERNAME': 1, 'i': 1, 'm': 1, 'so': 1, 'proud': 1})
{'USERNAME': 1, 'what': 1, 's': 1, 'happening': 1})
{'USERNAME': 1, 'unearthvii': 1, 'really': 1, 'masterpiece': 1, 'go': 1, 'fridge': 1, 'even': 1, 'kid': 1, 'like': 1, 'dang': 1, 's': 1, 'pretty': 1, 'good': 1})


In [21]:
from scipy.sparse import csr_matrix

class WordCounterToVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size=1000):
        self.vocabulary_size = vocabulary_size
    def fit(self, X, y=None):
        total_count = Counter()
        for word_count in X:
            for word, count in word_count.items():
                total_count[word] += min(count, 10)
        most_common = total_count.most_common()[:self.vocabulary_size]
        self.most_common_ = most_common
        self.vocabulary_ = {word: index + 1 for index, (word, count) in enumerate(most_common)}
        return self
    def transform(self, X, y=None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data, (rows, cols)), shape=(len(X), self.vocabulary_size + 1))

In [22]:
vocab_transformer = WordCounterToVectorTransformer()
X_vectors = vocab_transformer.fit_transform(X_wordcounts)
X_vectors

<779x1001 sparse matrix of type '<class 'numpy.int32'>'
	with 6476 stored elements in Compressed Sparse Row format>

In [23]:
len(vocab_transformer.vocabulary_)

1000

In [32]:
# from original data
X.loc[20:20, ]

,ID,username,tweet,index
20,1312949693404438528,defikigai,finding money 😃🤲🏻,516760


In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

preprocess_pipeline = Pipeline([
    ("document_to_wordcount", DocumentToWordCounterTransformer()),
    ("wordcount_to_vector", WordCounterToVectorTransformer()),
])

X_train_transformed = preprocess_pipeline.fit_transform(X_array)

In [34]:
X_train_transformed

<779x1001 sparse matrix of type '<class 'numpy.int32'>'
	with 6476 stored elements in Compressed Sparse Row format>

In [35]:
y_array = y.iloc[:,0].ravel()

In [36]:
from sklearn.model_selection import cross_val_score

log_clf = LogisticRegression(solver="liblinear", random_state=42)
score = cross_val_score(log_clf, X_train_transformed, y_array, cv=5, verbose=3, scoring='accuracy')
print('Mean accuracy: ' + str(score.mean()))

[CV]  ................................................................
[CV] .................................... , score=0.564, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.571, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.609, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.532, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.600, total=   0.0s
Mean accuracy: 0.5751282051282052


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


In [37]:
from sklearn.naive_bayes import MultinomialNB

NB_clf = MultinomialNB()
score = cross_val_score(NB_clf, X_train_transformed, y_array, cv=5, verbose=3, scoring='accuracy')
print('Mean accuracy: ' + str(score.mean()))

[CV]  ................................................................
[CV] .................................... , score=0.603, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.538, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.603, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.571, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.574, total=   0.0s
Mean accuracy: 0.5776592224979321


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished
